In [27]:
import pandas as pd
import numpy as np
import os
import pyarrow as pa
import pyarrow.parquet as pq

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

## Report summary example 
Used `coral_batch_eval.py` 


In [28]:
# Read in data/coral/counts/combined_df_with_responses.csv and drug counts
combined_df_with_responses = pd.read_csv(
    "../../data/coral/counts/combined_df_with_responses.csv"
)
drug_counts = pd.read_csv("../../data/coral/counts/drug_counts.csv")

combined_df_with_responses.head(1)

coral_idx     Sex                 UCSFDerivedRaceEthnicity_X   BirthDate  \
0          0  Female  Native Hawaiian or Other Pacific Islander  1964-03-25   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [30]:
drug_counts.head(20)

,Keyword,response_count,note_text_count
0,tamoxifen,2,2
1,gemcitabine,1,1
2,pembrolizumab,1,1
3,carboplatin,1,1
4,omeprazole,1,1
5,irinotecan,1,1
6,abraxane,1,1
7,pravastatin,1,1
8,letrozole,1,1
9,leuprolide,1,1


## Total counts old


In [ ]:
df_dir = "../../data/coral/post_edit/"

In [ ]:
def load_dataset(pre_filtered_df_dir, transformation):
    dataset_path = os.path.join(
        pre_filtered_df_dir,
        f"{transformation}.parquet",
    )

    if not os.path.exists(dataset_path):
        print(f"Dataset not found at path: {dataset_path}")
        return None

    return pd.read_parquet(dataset_path)


def merge_datasets_with_transformation(pre_filtered_df_dir):
    transformations = ["original", "brand_to_generic", "generic_to_brand"]
    merged_df = pd.DataFrame()

    for transformation in transformations:
        df = load_dataset(pre_filtered_df_dir, transformation)
        if df is not None:
            df["transformation_type"] = transformation
            df["question_index"] = range(len(df))
            merged_df = pd.concat([merged_df, df], ignore_index=True)

    return merged_df


merged_df = merge_datasets_with_transformation(df_dir)
merged_df = merged_df.sort_values(by=["coral_idx"])

if not merged_df.empty:
    print(f"\nMerged Dataset")
    print(f"Contains {len(merged_df)} rows.")
else:
    print(f"Failed to load and merge datasets for inspection.")


Merged Dataset
Contains 555 rows.


In [29]:
merged_df.head(1)

NameError: name 'merged_df' is not defined

In [ ]:
import os
import pandas as pd
from collections import Counter

# Assuming df_dir is defined and dataset_path is constructed correctly
dataset_path = os.path.join(df_dir, "original.parquet")

# Read the original dataset
original_df = pd.read_parquet(dataset_path)

# Flatten the list of keywords and count occurrences
all_keywords = [
    keyword for sublist in original_df["found_keywords"] for keyword in sublist
]
keyword_counts = Counter(all_keywords)

# Optionally, display the counts in a more readable format
keyword_counts_df = pd.DataFrame(keyword_counts.items(), columns=["Keyword", "Count"])
keyword_counts_df = keyword_counts_df.sort_values(by="Count", ascending=False)
print(keyword_counts_df)

# print number unique keywords
print(f"Number of unique keywords: {len(keyword_counts)}")

# Print the total count of keywords
total_keywords_count = sum(keyword_counts.values())
print(f"Total count of keywords: {total_keywords_count}")

                   Keyword  Count
4              gemcitabine     82
13              paclitaxel     50
11                abraxane     37
31               tamoxifen     34
5             capecitabine     33
34               letrozole     31
10              omeprazole     29
12                prilosec     27
2               irinotecan     26
6              oxaliplatin     24
54                   taxol     23
21                  xeloda     15
36                    dexa     15
18           dexamethasone     15
32                  femara     14
64               denosumab     13
45                arimidex     11
40             trastuzumab     11
15                decadron     11
52                 cytoxan     10
16             carboplatin     10
9               prednisone     10
46             amoxicillin     10
27          hydrocortisone     10
72              everolimus      9
47              exemestane      9
53             anastrozole      8
51                taxotere      8
82            

In [ ]:
# save original df to csv for inspection
original_df.to_csv("../../data/coral/post_edit/original.csv", index=False)